In [1]:
from mr_eval.utils.utils import *
from datasets import load_dataset
from collections import Counter
import random
from tqdm import tqdm

/mnt/petrelfs/songmingyang/anaconda3/envs/smoe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_answer_from_model_response(model_response):
    # 使用正则表达式匹配 \boxed{内容}，允许内容中包含大括号
    stack = []
    start = None
    for i, char in enumerate(model_response):
        if i < 7:
            continue
        if model_response[i-7:i] == '\\boxed{':
            stack.append('{')
            if start is None:
                start = i
        elif char == '{' and stack:
            stack.append('{')
        elif char == '}' and stack:
            stack.pop()
            if not stack:
                return model_response[start:i]
    return None


def get_most_common(lst):
    # 使用Counter统计每个元素出现次数
    count = Counter(lst)
    
    # 获取最大出现次数
    max_count = max(count.values())
    
    # 找出所有出现次数等于max_count的元素
    most_common = [item for item, freq in count.items() if freq == max_count]
    
    # 如果只有一个元素出现次数最多,返回该元素
    # 如果有多个元素出现次数相同,随机返回一个
    return random.choice(most_common)

def compare_fraction_decimal(str1, str2):
    def convert_to_float(s):
        # 处理分数形式 \frac{a}{b}
        if r'\\frac' in s:
            # 提取分子分母
            num = s[s.find('{')+1:s.find('}')]
            denom = s[s.rfind('{')+1:s.rfind('}')]
            return float(num) / float(denom)
        else:
            # 处理普通数字字符串
            return float(s)
    
    try:
        num1 = convert_to_float(str1)
        num2 = convert_to_float(str2)
        # 使用近似相等来处理浮点数精度问题
        return abs(num1 - num2) < 1e-10
    except:
        return False

def get_clean_str(ans):
    if ans is None:
        return None
    ans = ans.replace("//","").replace(" ","").replace("dfrac","frac")
    ans = ans.replace("\\","")
    ans = ans.replace("dfrac","frac")
    ans = ans.replace("\\left","").replace("\\right","")
    ans = ans.replace("\\begin{matrix}","\\begin{pmatrix}").replace("\\end{matrix}","\\end{pmatrix}")
    ans = ans.replace("\\begin{vmatrix}","\\begin{pmatrix}").replace("\\end{vmatrix}","\\end{pmatrix}")
    ans = ans.replace("\\begin{bmatrix}","\\begin{pmatrix}").replace("\\end{bmatrix}","\\end{pmatrix}")
    ans = ans.replace("\\quad","").replace("\\;","").replace("\\,","")
    ans = ans.replace("^{\\prime}","'")
    ans = ans.replace("\\cdot","*")
    ans = ans.replace("\\times","*")
    ans = ans.replace("{\\rm ","").replace("}","")
    ans = ans.replace("\\text{","").replace("}","")
    return ans


def compare_answer(ans1,ans2):
    if ans1 is None or ans2 is None:
        return False
    assert isinstance(ans1,str) and isinstance(ans2,str), f"ans1: {ans1}, ans2: {ans2}"
    
    if ans1 in ans2 or ans2 in ans1:
        return True
    return False

In [4]:
## GSM8K 
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k"
# dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/MATH"
dataset = load_dataset(dataset_addr,"main")
dataset["test"][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [3]:
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_subset200/gsm8k.jsonl"
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/gsm8k.jsonl"
inferenced_data = process_jsonl(inferenced_data)
inferenced_data[0]


{'id': 'gsm8k_0',
 'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': '18',
 'responses': ["Alright, so I've got this problem about Janet and her ducks laying eggs. Let's break it down step by step.\n\nFirst, it says that Janet’s ducks lay 16 eggs per day. So, every day, she gets 16 eggs.\n\nNext, she eats three eggs for breakfast every morning. So, from the 16 eggs, she consumes 3 herself. That means the number of eggs left after she eats is:\n\n16 eggs - 3 eggs = 13 eggs\n\nThen, she bakes muffins for her friends every day and uses four eggs for that. So, from the remaining 13 eggs, she uses 4 for baking. That leaves:\n\n13 eggs - 4 eggs = 9 eggs\n\nNow, she sells the remainder at the farmers' market daily for $2 per fresh duck egg.

In [ ]:
for item in inferenced_data:
    ground_truth = item["answer"]
    labels = []
    for i, response in enumerate(item["responses"]):
        pred_str = extract_answer_from_model_response(response)
        pred_str = get_clean_str(pred_str) if pred_str else None
        ground_truth_str = get_clean_str(ground_truth) if ground_truth else None
        correct = compare_answer(pred_str, ground_truth_str)
        labels.append(correct)
    item["labels"] = labels

In [4]:
## Math

inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_subset200/math.jsonl"
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/math.jsonl"
inferenced_data = process_jsonl(inferenced_data)
inferenced_data[0]

{'id': 'math_0',
 'question': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?',
 'answer': '2',
 'responses': ["So I've got this function: y = 2 / (x² + x - 6). I need to find out how many vertical asymptotes its graph has. I remember that vertical asymptotes occur where the function goes up or down towards infinity, which happens when the denominator is zero, as long as the numerator isn't zero at those points.\n\nFirst, I should find where the denominator equals zero because that's where the function might have vertical asymptotes. The denominator is x² + x - 6. I need to solve the equation x² + x - 6 = 0 to find the x-values that make the denominator zero.\n\nLet me try factoring the quadratic. I'm looking for two numbers that multiply to -6 and add up to 1 (the coefficient of x). Let's see, 3 and -2: 3 * -2 = -6, and 3 + (-2) = 1. Perfect!\n\nSo, x² + x - 6 = (x + 3)(x - 2). Setting this equal to zero gives me:\n\n(x + 3)(x - 2) = 0\n\nWhich means x + 3

In [5]:
for item in inferenced_data:
    answer = extract_answer_from_model_response(item["ans_str"])
    item["answer"] = answer

write_jsonl(inferenced_data,"/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_full/math.jsonl")

In [6]:
## olympiadbench
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_subset200/olympiadbench.jsonl"
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/olympiadbench.jsonl"
inferenced_data = process_jsonl(inferenced_data)
inferenced_data[3]

{'id': 'olympiadbench_3',
 'question': 'Determine all positive integers $n$ satisfying the following condition: for every monic polynomial $P$ of degree at most $n$ with integer coefficients, there exists a positive integer $k \\leq n$, and $k+1$ distinct integers $x_{1}, x_{2}, \\ldots, x_{k+1}$ such that\n\n\n\n$$\n\nP\\left(x_{1}\\right)+P\\left(x_{2}\\right)+\\cdots+P\\left(x_{k}\\right)=P\\left(x_{k+1}\\right) .\n\n$$\n\n\nNote. A polynomial is monic if the coefficient of the highest power is one.',
 'answer': '2',
 'responses': ['So I have this problem here that I need to solve. It\'s about finding all positive integers \\( n \\) with a certain property related to monic polynomials with integer coefficients. The condition is that for every monic polynomial \\( P \\) of degree at most \\( n \\), there should exist a positive integer \\( k \\leq n \\) and \\( k+1 \\) distinct integers \\( x_1, x_2, \\ldots, x_{k+1} \\) such that the sum of the polynomial evaluated at the first \\( 

In [7]:
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/gsm8k"
dataset_addr = "/mnt/petrelfs/songmingyang/songmingyang/data/reasoning/OlympiadBench"
configs = ['OE_MM_maths_en_COMP', 'OE_MM_maths_zh_CEE', 'OE_MM_maths_zh_COMP', 'OE_MM_physics_en_COMP', 'OE_MM_physics_zh_CEE', 'OE_TO_maths_en_COMP', 'OE_TO_maths_zh_CEE', 'OE_TO_maths_zh_COMP', 'OE_TO_physics_en_COMP', 'OE_TO_physics_zh_CEE', 'TP_MM_maths_en_COMP', 'TP_MM_maths_zh_CEE', 'TP_MM_maths_zh_COMP', 'TP_MM_physics_en_COMP', 'TP_TO_maths_en_COMP', 'TP_TO_maths_zh_CEE', 'TP_TO_maths_zh_COMP', 'TP_TO_physics_en_COMP']
target_item = None
target_list = []
for config in configs:
    dataset = load_dataset(dataset_addr,config)
    for item in dataset["train"]:
        if item["modality"] != "Multimodal":
            target_list.append(item)

len(target_list)

3607

In [8]:
target_list[0]

{'id': 1606,
 'question': 'Xenia and Sergey play the following game. Xenia thinks of a positive integer $N$ not exceeding 5000. Then she fixes 20 distinct positive integers $a_{1}, a_{2}, \\ldots, a_{20}$ such that, for each $k=1,2, \\ldots, 20$, the numbers $N$ and $a_{k}$ are congruent modulo $k$. By a move, Sergey tells Xenia a set $S$ of positive integers not exceeding 20 , and she tells him back the set $\\left\\{a_{k}: k \\in S\\right\\}$ without spelling out which number corresponds to which index. How many moves does Sergey need to determine for sure the number Xenia thought of?',
 'solution': ["Sergey can determine Xenia's number in 2 but not fewer moves.\n\n\n\nWe first show that 2 moves are sufficient. Let Sergey provide the set $\\{17,18\\}$ on his first move, and the set $\\{18,19\\}$ on the second move. In Xenia's two responses, exactly one number occurs twice, namely, $a_{18}$. Thus, Sergey is able to identify $a_{17}, a_{18}$, and $a_{19}$, and thence the residue of $N$

In [9]:
for item in inferenced_data:
    idx = item["id"]
    origin_idx = idx.split("_")[1].strip()
    origin_idx = int(origin_idx)
    target_item = target_list[origin_idx]
    assert target_item["question"] == item["question"]
    item["answer"] = target_item["final_answer"]
    item["ans_str"] = target_item["solution"]

In [11]:
inferenced_data[2]

{'id': 'olympiadbench_2',
 'question': '\nFind (in closed form) the difference between the number of positive integers at most $2^{2017}$ with even weight and the number of positive integers at most $2^{2017}$ with odd weight.',
 'answer': ['$2^{1009}$'],
 'responses': ['So I have this problem here: I need to find the difference between the number of positive integers at most \\(2^{2017}\\) with even weight and those with odd weight. Now, I\'m a bit confused about what "weight" means in this context. I think it refers to the number of 1\'s in the binary representation of the number, also known as the Hamming weight. So, for example, the number 3 in binary is 11, which has a Hamming weight of 2, which is even, and the number 2 is 10 in binary, with a Hamming weight of 1, which is odd.\n\nAlright, so the problem is asking for the difference between the count of numbers up to \\(2^{2017}\\) that have an even number of 1\'s in their binary representation and those that have an odd number o

In [12]:
write_jsonl(inferenced_data,"/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_full/olympiadbench.jsonl")

In [13]:
### mmlu
inferenced_data = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/inference_res_full/mmlu.jsonl"
inferenced_data = process_jsonl(inferenced_data)
inferenced_data[0]

{'id': 'mmlu_0',
 'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'answer': '4',
 'responses': ["So I have this problem here: find the degree of the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. Alright, I need to find [Q(sqrt(2), sqrt(3), sqrt(18)) : Q]. I'm a bit rusty on field extensions, but let's try to work through this step by step.\n\nFirst, I know that when we have a field extension like Q(sqrt(2)), the degree is 2 because sqrt(2) is a root of the polynomial x^2 - 2, which is irreducible over Q. Similarly, Q(sqrt(3)) has degree 2 over Q since sqrt(3) is a root of x^2 - 3, which is also irreducible over Q.\n\nNow, in this problem, we have three square roots: sqrt(2), sqrt(3), and sqrt(18). But wait a minute, sqrt(18) can be simplified. sqrt(18) = sqrt(9*2) = 3*sqrt(2). So, sqrt(18) is just 3 times sqrt(2). That means Q(sqrt(2), sqrt(3), sqrt(18)) is actually the same as Q(sqrt(2), sqrt(3), 3*sqrt(2)). But 3*sqrt(2) is 

In [14]:
alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
renewd_data = []
for item in inferenced_data:
    question = item["question"]
    choices = item["choices"]
    new_question = f"{question} Please select the correct answer from the following options: \n"
    for i, choice in enumerate(choices):
        new_question += f"{alphabet[i]}. {choice}\n"
    
    ans_index = item["ans_str"]
    ans_str = f"{alphabet[ans_index]}. {choices[ans_index]}"
    answer = f"{alphabet[ans_index]}"

    item["question"] = new_question
    item["answer"] = answer
    item["ans_str"] = ans_str
    renewd_data.append(item)

In [15]:
write_jsonl(renewd_data,"/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/mmlu_full_input.jsonl")

In [16]:
renewd_data[0]

{'id': 'mmlu_0',
 'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. Please select the correct answer from the following options: \nA. 0\nB. 4\nC. 2\nD. 6\n',
 'answer': 'B',
 'responses': ["So I have this problem here: find the degree of the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. Alright, I need to find [Q(sqrt(2), sqrt(3), sqrt(18)) : Q]. I'm a bit rusty on field extensions, but let's try to work through this step by step.\n\nFirst, I know that when we have a field extension like Q(sqrt(2)), the degree is 2 because sqrt(2) is a root of the polynomial x^2 - 2, which is irreducible over Q. Similarly, Q(sqrt(3)) has degree 2 over Q since sqrt(3) is a root of x^2 - 3, which is also irreducible over Q.\n\nNow, in this problem, we have three square roots: sqrt(2), sqrt(3), and sqrt(18). But wait a minute, sqrt(18) can be simplified. sqrt(18) = sqrt(9*2) = 3*sqrt(2). So, sqrt(18) is just 3 times sqrt(2). That means Q(sqrt(2), sqrt

In [24]:
mmlu_new = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_res200/mmluChoice.jsonl"
mmlu_new = process_jsonl(mmlu_new)
mmlu_new[0]

{'id': 'mmluChoice_387',
 'question': '_______ such as bitcoin are becoming increasingly mainstream and have a whole host of associated ethical implications, for example, they are______ and more ______. However, they have also been used to engage in _______. Please select the correct answer from the following options: \nA. Cryptocurrencies, Expensive, Secure, Financial Crime\nB. Traditional currency, Cheap, Unsecure, Charitable giving\nC. Cryptocurrencies, Cheap, Secure, Financial crime\nD. Traditional currency, Expensive, Unsecure, Charitable giving\n',
 'answer': 'C',
 'responses': ['Alright, so I\'ve got this question here about cryptocurrencies and their ethical implications. It\'s asking me to choose the correct options from a list, and it seems like it\'s testing my understanding of what cryptocurrencies are and their impacts. Let me read the question again to make sure I get it.\n\n"_______ such as bitcoin are becoming increasingly mainstream and have a whole host of associated 

In [18]:
import random
input_path = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_full/mmluChoice.jsonl"
output_path = "/mnt/petrelfs/songmingyang/code/reasoning/MR_Hallucination/mr_eval/bon_eval/scripts/logs/fix_inference_res3000/mmluChoice.jsonl"
input_data = process_jsonl(input_path)
# output_data = random.sample(input_data,3000)
len(input_data)

2900

In [ ]:
write_jsonl(output_data,output_path)